# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [41]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [42]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atuona,-9.8000,-139.0333,26.07,76,19,6.78,PF,1675817481
1,1,upernavik,72.7868,-56.1549,-29.63,97,60,1.47,GL,1675817481
2,2,busselton,-33.6500,115.3333,22.42,57,0,3.71,AU,1675817482
3,3,kjopsvik,68.0970,16.3742,2.93,85,100,6.46,NO,1675817483
4,4,geraldton,-28.7667,114.6000,24.23,64,0,6.17,AU,1675817472


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 500,
    size = "Humidity",
    scale = 0.70,
    color = "City",
    alpha=0.5
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [44]:
# Narrow down cities that fit criteria and drop any results with null values
#ideal_wheather = city_data_df.query("`Max Temp` >= 21 and `Max Temp` <=26")
#ideal_wheather = ideal_wheather.query("Humidity < 25 and Cloudiness < 40")

# Drop any rows with null values
ideal_wheather = city_data_df.dropna()

# Display sample data
ideal_wheather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atuona,-9.8000,-139.0333,26.07,76,19,6.78,PF,1675817481
1,1,upernavik,72.7868,-56.1549,-29.63,97,60,1.47,GL,1675817481
2,2,busselton,-33.6500,115.3333,22.42,57,0,3.71,AU,1675817482
3,3,kjopsvik,68.0970,16.3742,2.93,85,100,6.46,NO,1675817483
4,4,geraldton,-28.7667,114.6000,24.23,64,0,6.17,AU,1675817472
...,...,...,...,...,...,...,...,...,...,...
564,564,adre,13.4667,22.2000,18.35,24,46,9.06,TD,1675817898
565,565,vrangel,42.7297,133.0839,-4.85,51,4,3.19,RU,1675817899
566,566,grindavik,63.8424,-22.4338,-3.03,100,75,13.38,IS,1675817899
567,567,juruti,-2.1522,-56.0922,23.83,94,100,0.97,BR,1675817899


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(city_data_df[["City", "Country","Lat" ,"Lng", "Humidity"]])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:, "Hotel Name"] = None

# Display sample data
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        569 non-null    object 
 1   Country     565 non-null    object 
 2   Lat         569 non-null    float64
 3   Lng         569 non-null    float64
 4   Humidity    569 non-null    int64  
 5   Hotel Name  0 non-null      object 
dtypes: float64(2), int64(1), object(3)
memory usage: 26.8+ KB


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit": 1,
    "filter": None,
    "bias": None,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row.loc["Lat"]
    longitude = row.loc["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
atuona - nearest hotel: Pearl Resort
upernavik - nearest hotel: No hotel found
busselton - nearest hotel: Broadwater Beach Resort
kjopsvik - nearest hotel: Stetind Hotel
geraldton - nearest hotel: Ocean Centre Hotel
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
acapulco - nearest hotel: Hotel del Valle
isangel - nearest hotel: Tanna Lodge
chuy - nearest hotel: Alerces
vaini - nearest hotel: Keleti International Resort
kavieng - nearest hotel: Kavieng Hotel
hami - nearest hotel: Хами
dalnegorsk - nearest hotel: Дальнегорск
hobart - nearest hotel: Vibe Hotel Hobart
sao filipe - nearest hotel: Bela Vista
nikolskoye - nearest hotel: No hotel found
honningsvag - nearest hotel: Scandic Honningsvåg
punta arenas - nearest hotel: Hotel Lacolet
tenenkou - nearest hotel: No hotel found
moron - nearest hotel: La Esquina
havre-saint-pierre - nearest hotel: Complexe MV
tutoia - nearest hotel: Pousada San Vicente
nizhniy odes - nearest hotel: No hotel found
jamestown - n

coihaique - nearest hotel: No hotel found
chokurdakh - nearest hotel: No hotel found
esperance - nearest hotel: Hospitality
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
saint-philippe - nearest hotel: Le Baril
port hardy - nearest hotel: Quarterdeck Inn
susurluk - nearest hotel: No hotel found
coahuayana - nearest hotel: No hotel found
touros - nearest hotel: Pousada Atlântico
cabo san lucas - nearest hotel: Comfort Rooms
sao raimundo nonato - nearest hotel: Pousada Zabelê
talcahuano - nearest hotel: Club Recreativo Armada de Chile
kijang - nearest hotel: 힐튼호텔 부산
solnechnyy - nearest hotel: Заря
valparaiso - nearest hotel: Ibis Hotel
aksaray - nearest hotel: Grand Saatcioglu Otel
gimli - nearest hotel: Gimli Hotel
nacogdoches - nearest hotel: No hotel found
idenao - nearest hotel: No hotel found
dingle - nearest hotel: Hotel Edmundo
pimentel - nearest hotel: Hospedaje Calle del Sol
vardo - nearest hotel: Meieriet
new norfolk - nearest hotel: Woodbridge on the D

tornio - nearest hotel: Park Hotel Tornio
muros - nearest hotel: Pensión Ría de Muros
tongren - nearest hotel: 闽 商大 酒 店
grand island - nearest hotel: Grand Island Travelodge
flin flon - nearest hotel: Copperbelt Hotel
at-bashi - nearest hotel: Tash Rabat
north platte - nearest hotel: Howard Johnson Inn - North Platte
krasnokholm - nearest hotel: No hotel found
petropavlovsk-kamchatskiy - nearest hotel: ООО Постоялый двор
qaqortoq - nearest hotel: Hotel Qaqortoq
grand bend - nearest hotel: No hotel found
lawrence - nearest hotel: The Eldridge Hotel
barra do garcas - nearest hotel: Pousada Portal
puerto montt - nearest hotel: Le Mirage
alofi - nearest hotel: Matavai Resort
eyl - nearest hotel: فندق ايل دابي
moreno valley - nearest hotel: Hotel XOLA Trademark Collection by Wyndham
itarema - nearest hotel: Pousada Oásis
itupiranga - nearest hotel: No hotel found
balfour - nearest hotel: No hotel found
tevaitoa - nearest hotel: Chambre d'hote
limay - nearest hotel: Acacia Lane
puerto salgar

,City,Country,Lat,Lng,Humidity,Hotel Name
0,atuona,PF,-9.8000,-139.0333,76,Pearl Resort
1,upernavik,GL,72.7868,-56.1549,97,No hotel found
2,busselton,AU,-33.6500,115.3333,57,Broadwater Beach Resort
3,kjopsvik,NO,68.0970,16.3742,85,Stetind Hotel
4,geraldton,AU,-28.7667,114.6000,64,Ocean Centre Hotel
...,...,...,...,...,...,...
564,adre,TD,13.4667,22.2000,24,No hotel found
565,vrangel,RU,42.7297,133.0839,51,No hotel found
566,grindavik,IS,63.8424,-22.4338,100,The Retreat at Blue Lagoon Iceland
567,juruti,BR,-2.1522,-56.0922,94,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [47]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 500,
    size = "Humidity",
    scale = 0.70,
    color = "City",
    alpha=0.5,
    hover_cols='all'
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)